In [259]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

#### Read table from annex in Wikipedia

In [260]:
df=pd.read_excel(r"C:\\Users\caika\OneDrive\Desktop\pYTHON\DATA LOCATION\barriosbsas_.xlsx")

In [261]:
df.head()

,column1,Neighbourhoods,Area,Population,District
0,0,"Agronomía, Ciudad de Buenos Aires, Argentina",2.1,34580,15
1,1,"Almagro, Ciudad de Buenos Aires, Argentina",4.1,139262,5
2,2,"Balvanera, Ciudad de Buenos Aires, Argentina",4.4,152198,3
3,3,"Barracas, Ciudad de Buenos Aires, Argentina",7.6,77474,4
4,4,"Belgrano, Ciudad de Buenos Aires, Argentina",6.8,138942,13


In [262]:
df.shape

(48, 5)

In [263]:
df.drop('column1', inplace=True, axis=1)
df.head()

,Neighbourhoods,Area,Population,District
0,"Agronomía, Ciudad de Buenos Aires, Argentina",2.1,34580,15
1,"Almagro, Ciudad de Buenos Aires, Argentina",4.1,139262,5
2,"Balvanera, Ciudad de Buenos Aires, Argentina",4.4,152198,3
3,"Barracas, Ciudad de Buenos Aires, Argentina",7.6,77474,4
4,"Belgrano, Ciudad de Buenos Aires, Argentina",6.8,138942,13


#### Looking for coordinates of the neighborhoods

In [264]:
address = 'Ciudad de Buenos Aires, Argentina'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ciudad de Buenos Aires are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ciudad de Buenos Aires are -34.6075682, -58.4370894.


In [265]:
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 
   
# declare an empty list to store 
# latitude and longitude of values  
# of city column 
longitude = [] 
latitude = [] 
   
# function to find the coordinate 
# of a given city  
def findGeocode(city): 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="explorer") 
          
        return geolocator.geocode(city) 
      
    except GeocoderTimedOut: 
          
        return findGeocode(city)     
  
# each value from city column 
# will be fetched and sent to 
# function find_geocode    
for i in (df["Neighbourhoods"]): 
      
    if findGeocode(i) != None: 
           
        loc = findGeocode(i) 
          
        # coordinates returned from  
        # function is stored into 
        # two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else: 
        latitude.append(np.nan) 
        longitude.append(np.nan)

In [266]:
df["Longitude"] = longitude 
df["Latitude"] = latitude

In [267]:
print(df)

                                       Neighbourhoods  Area  Population  \
0        Agronomía, Ciudad de Buenos Aires, Argentina   2.1       34580   
1          Almagro, Ciudad de Buenos Aires, Argentina   4.1      139262   
2        Balvanera, Ciudad de Buenos Aires, Argentina   4.4      152198   
3         Barracas, Ciudad de Buenos Aires, Argentina   7.6       77474   
4         Belgrano, Ciudad de Buenos Aires, Argentina   6.8      138942   
5            Boedo, Ciudad de Buenos Aires, Argentina   2.6       48520   
6        Caballito, Ciudad de Buenos Aires, Argentina   6.8      183396   
7        Chacarita, Ciudad de Buenos Aires, Argentina   3.1       27440   
8          Coghlan, Ciudad de Buenos Aires, Argentina   1.3       19177   
9       Colegiales, Ciudad de Buenos Aires, Argentina   2.3       56998   
10    Constitución, Ciudad de Buenos Aires, Argentina   2.1       45860   
11          Flores, Ciudad de Buenos Aires, Argentina   7.8      150484   
12        Floresta, Ciuda

In [268]:
df.dropna(inplace=True)

In [269]:
df.shape

(47, 6)

#### Connection to Foursquare

In [270]:
CLIENT_ID = 'A0AM453CCFJ4AXYHB004LNJDY2GDU2UE2BVUMUZB2BA4KIN5' # your Foursquare ID
CLIENT_SECRET = 'KIPE3R01LXDJXHXXRXLW2IHKRSC4DMKPLWWMXQXOOPMJXGRZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A0AM453CCFJ4AXYHB004LNJDY2GDU2UE2BVUMUZB2BA4KIN5
CLIENT_SECRET:KIPE3R01LXDJXHXXRXLW2IHKRSC4DMKPLWWMXQXOOPMJXGRZ


In [271]:
neighbourhood_latitude = df.loc[21, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df.loc[21, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df.loc[21, 'Neighbourhoods'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Palermo, Ciudad de Buenos Aires, Argentina are -34.5803362, -58.4245236.


In [272]:
radius=1000
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighbourhood_latitude, 
            neighbourhood_longitude, 
            radius, 
            LIMIT)

results = requests.get(url).json()
results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-272-d8c87410400c>:29: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Megatlon,Gym,-34.579216,-58.426123
1,Nicolo Helados,Ice Cream Shop,-34.581652,-58.426735
2,La Carnicería,Argentinian Restaurant,-34.582814,-58.423778
3,Distrito Arcos,Shopping Mall,-34.580494,-58.427621
4,Vain Boutique Hotel Buenos Aires,Hotel,-34.583672,-58.424980


In [273]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [274]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [275]:
bsas_venues = getNearbyVenues(names=df['Neighbourhoods'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Agronomía, Ciudad de Buenos Aires, Argentina
Almagro, Ciudad de Buenos Aires, Argentina
Balvanera, Ciudad de Buenos Aires, Argentina
Barracas, Ciudad de Buenos Aires, Argentina
Belgrano, Ciudad de Buenos Aires, Argentina
Boedo, Ciudad de Buenos Aires, Argentina
Caballito, Ciudad de Buenos Aires, Argentina
Chacarita, Ciudad de Buenos Aires, Argentina
Coghlan, Ciudad de Buenos Aires, Argentina
Colegiales, Ciudad de Buenos Aires, Argentina
Constitución, Ciudad de Buenos Aires, Argentina
Flores, Ciudad de Buenos Aires, Argentina
Floresta, Ciudad de Buenos Aires, Argentina
La Boca, Ciudad de Buenos Aires, Argentina
La Paternal, Ciudad de Buenos Aires, Argentina
Liniers, Ciudad de Buenos Aires, Argentina
Mataderos, Ciudad de Buenos Aires, Argentina
Montserrat, Ciudad de Buenos Aires, Argentina
Monte Castro, Ciudad de Buenos Aires, Argentina
Nueva Pompeya, Ciudad de Buenos Aires, Argentina
Núñez, Ciudad de Buenos Aires, Argentina
Palermo, Ciudad de Buenos Aires, Argentina
Parque Avellaneda, C

In [276]:
bsas_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Agronomía, Ciudad de Buenos Aires, Argentina",26,26,26,26,26,26
"Almagro, Ciudad de Buenos Aires, Argentina",100,100,100,100,100,100
"Balvanera, Ciudad de Buenos Aires, Argentina",100,100,100,100,100,100
"Barracas, Ciudad de Buenos Aires, Argentina",15,15,15,15,15,15
"Belgrano, Ciudad de Buenos Aires, Argentina",100,100,100,100,100,100
"Boedo, Ciudad de Buenos Aires, Argentina",51,51,51,51,51,51
"Caballito, Ciudad de Buenos Aires, Argentina",100,100,100,100,100,100
"Chacarita, Ciudad de Buenos Aires, Argentina",78,78,78,78,78,78
"Coghlan, Ciudad de Buenos Aires, Argentina",61,61,61,61,61,61


In [277]:
print('There are {} uniques categories.'.format(len(bsas_venues['Venue Category'].unique())))

There are 248 uniques categories.


In [278]:
# one hot encoding
bsas_onehot = pd.get_dummies(bsas_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
bsas_onehot['Neighbourhood'] = bsas_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [bsas_onehot.columns[-1]] + list(bsas_onehot.columns[:-1])
bsas_onehot = bsas_onehot[fixed_columns]

bsas_onehot

,Neighbourhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Agronomía, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Agronomía, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Agronomía, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Agronomía, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Agronomía, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2761,"Villa Urquiza, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2762,"Villa Urquiza, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2763,"Villa Urquiza, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2764,"Villa Urquiza, Ciudad de Buenos Aires, Argentina",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [279]:
for col in bsas_onehot.columns: 
    print(col)

Neighbourhood
Accessories Store
Adult Boutique
African Restaurant
American Restaurant
Amphitheater
Arcade
Argentinian Restaurant
Art Gallery
Art Museum
Arts & Crafts Store
Arts & Entertainment
Asian Restaurant
Athletics & Sports
Auto Garage
Auto Workshop
BBQ Joint
Bagel Shop
Bakery
Bar
Basketball Court
Basketball Stadium
Bed & Breakfast
Beer Bar
Beer Garden
Beer Store
Big Box Store
Bike Rental / Bike Share
Bike Shop
Bistro
Boarding House
Boat or Ferry
Bookstore
Botanical Garden
Boutique
Bowling Alley
Breakfast Spot
Brewery
Bridal Shop
Buffet
Burger Joint
Bus Station
Bus Stop
Business Service
Café
Cajun / Creole Restaurant
Camera Store
Cheese Shop
Chinese Restaurant
Chocolate Shop
Church
Circus
Clothing Store
Cocktail Bar
Coffee Shop
College Football Field
Comedy Club
Comfort Food Restaurant
Comic Shop
Community Center
Concert Hall
Convenience Store
Cosmetics Shop
Creperie
Cultural Center
Dance Studio
Deli / Bodega
Department Store
Design Studio
Dessert Shop
Diner
Dive Bar
Eastern Europ

In [280]:
bsas_grouped = bsas_onehot.groupby('Neighbourhood').mean().reset_index()
bsas_grouped

,Neighbourhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Agronomía, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00
1,"Almagro, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.110000,0.000000,0.000000,...,0.020000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00
2,"Balvanera, Ciudad de Buenos Aires, Argentina",0.00,0.01,0.00,0.000000,0.00,0.00,0.020000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.010000,0.000000,0.00
3,"Barracas, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00
4,"Belgrano, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.020000,0.000000,0.010000,...,0.020000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.010000,0.000000,0.01
5,"Boedo, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.254902,0.019608,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00
6,"Caballito, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.040000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00
7,"Chacarita, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.038462,0.000000,0.000000,...,0.012821,0.012821,0.00,0.00,0.00,0.012821,0.00,0.000000,0.000000,0.00
8,"Coghlan, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.081967,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00
9,"Colegiales, Ciudad de Buenos Aires, Argentina",0.00,0.00,0.00,0.000000,0.00,0.00,0.090000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00


#### Let's only show columns for cafés

In [281]:
cols = ['Neighbourhood','Bakery','Breakfast Spot','Café','Coffee Shop','Diner','Pie Shop','Tea Room']
bsas_grouped_coffee = bsas_grouped[cols]

In [282]:
bsas_grouped_coffee

,Neighbourhood,Bakery,Breakfast Spot,Café,Coffee Shop,Diner,Pie Shop,Tea Room
0,"Agronomía, Ciudad de Buenos Aires, Argentina",0.000000,0.000000,0.038462,0.076923,0.000000,0.000000,0.000000
1,"Almagro, Ciudad de Buenos Aires, Argentina",0.010000,0.010000,0.050000,0.030000,0.000000,0.000000,0.000000
2,"Balvanera, Ciudad de Buenos Aires, Argentina",0.020000,0.010000,0.090000,0.040000,0.000000,0.000000,0.000000
3,"Barracas, Ciudad de Buenos Aires, Argentina",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Belgrano, Ciudad de Buenos Aires, Argentina",0.040000,0.020000,0.070000,0.070000,0.000000,0.000000,0.020000
5,"Boedo, Ciudad de Buenos Aires, Argentina",0.039216,0.019608,0.078431,0.019608,0.000000,0.000000,0.000000
6,"Caballito, Ciudad de Buenos Aires, Argentina",0.060000,0.000000,0.100000,0.100000,0.000000,0.000000,0.010000
7,"Chacarita, Ciudad de Buenos Aires, Argentina",0.051282,0.000000,0.064103,0.025641,0.000000,0.000000,0.000000
8,"Coghlan, Ciudad de Buenos Aires, Argentina",0.081967,0.000000,0.049180,0.000000,0.000000,0.000000,0.016393
9,"Colegiales, Ciudad de Buenos Aires, Argentina",0.060000,0.010000,0.040000,0.060000,0.000000,0.000000,0.000000


#### Filtering categoires of coffee places

In [283]:
bsas_grouped_coffee['COFFEES'] = bsas_grouped_coffee['Bakery'] + bsas_grouped_coffee['Breakfast Spot']+ bsas_grouped_coffee['Café']+ bsas_grouped_coffee['Coffee Shop'] + bsas_grouped_coffee['Diner']+ bsas_grouped_coffee['Pie Shop'] + bsas_grouped_coffee['Tea Room']
bsas_grouped_coffee = bsas_grouped_coffee[['Neighbourhood','COFFEES']]
bsas_grouped_coffee.head()

<ipython-input-283-f343b0d71b83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bsas_grouped_coffee['COFFEES'] = bsas_grouped_coffee['Bakery'] + bsas_grouped_coffee['Breakfast Spot']+ bsas_grouped_coffee['Café']+ bsas_grouped_coffee['Coffee Shop'] + bsas_grouped_coffee['Diner']+ bsas_grouped_coffee['Pie Shop'] + bsas_grouped_coffee['Tea Room']


,Neighbourhood,COFFEES
0,"Agronomía, Ciudad de Buenos Aires, Argentina",0.115385
1,"Almagro, Ciudad de Buenos Aires, Argentina",0.100000
2,"Balvanera, Ciudad de Buenos Aires, Argentina",0.160000
3,"Barracas, Ciudad de Buenos Aires, Argentina",0.000000
4,"Belgrano, Ciudad de Buenos Aires, Argentina",0.220000


In [284]:
df = df.join(bsas_grouped_coffee['COFFEES'])
df.head()

,Neighbourhoods,Area,Population,District,Longitude,Latitude,COFFEES
0,"Agronomía, Ciudad de Buenos Aires, Argentina",2.1,34580,15,-58.485385,-34.591516,0.115385
1,"Almagro, Ciudad de Buenos Aires, Argentina",4.1,139262,5,-58.422233,-34.609988,0.100000
2,"Balvanera, Ciudad de Buenos Aires, Argentina",4.4,152198,3,-58.403140,-34.609215,0.160000
3,"Barracas, Ciudad de Buenos Aires, Argentina",7.6,77474,4,-58.387562,-34.645285,0.000000
4,"Belgrano, Ciudad de Buenos Aires, Argentina",6.8,138942,13,-58.456545,-34.561308,0.220000


#### Add column with density of population

In [285]:
df['PopDensity'] = df['Population'] / (df['Area'] * 1000000)
df.head()

,Neighbourhoods,Area,Population,District,Longitude,Latitude,COFFEES,PopDensity
0,"Agronomía, Ciudad de Buenos Aires, Argentina",2.1,34580,15,-58.485385,-34.591516,0.115385,0.016467
1,"Almagro, Ciudad de Buenos Aires, Argentina",4.1,139262,5,-58.422233,-34.609988,0.100000,0.033966
2,"Balvanera, Ciudad de Buenos Aires, Argentina",4.4,152198,3,-58.403140,-34.609215,0.160000,0.034590
3,"Barracas, Ciudad de Buenos Aires, Argentina",7.6,77474,4,-58.387562,-34.645285,0.000000,0.010194
4,"Belgrano, Ciudad de Buenos Aires, Argentina",6.8,138942,13,-58.456545,-34.561308,0.220000,0.020433


#### Add column with coffee shops over Density of Population

In [286]:
df['CoffeeDensity'] = df['COFFEES'] / df['PopDensity']
df.head()

,Neighbourhoods,Area,Population,District,Longitude,Latitude,COFFEES,PopDensity,CoffeeDensity
0,"Agronomía, Ciudad de Buenos Aires, Argentina",2.1,34580,15,-58.485385,-34.591516,0.115385,0.016467,7.007163
1,"Almagro, Ciudad de Buenos Aires, Argentina",4.1,139262,5,-58.422233,-34.609988,0.100000,0.033966,2.944091
2,"Balvanera, Ciudad de Buenos Aires, Argentina",4.4,152198,3,-58.403140,-34.609215,0.160000,0.034590,4.625554
3,"Barracas, Ciudad de Buenos Aires, Argentina",7.6,77474,4,-58.387562,-34.645285,0.000000,0.010194,0.000000
4,"Belgrano, Ciudad de Buenos Aires, Argentina",6.8,138942,13,-58.456545,-34.561308,0.220000,0.020433,10.767083


In [290]:
df['CoffeeDensity']=df['CoffeeDensity'].round(2)
df.dropna(inplace=True)
df.sort_values(by=['CoffeeDensity'], ascending=True, inplace=True)
df

,Neighbourhoods,Area,Population,District,Longitude,Latitude,COFFEES,PopDensity,CoffeeDensity
19,"Nueva Pompeya, Ciudad de Buenos Aires, Argentina",6.2,63276,4,-58.414710,-34.652630,0.000000,0.010206,0.00
3,"Barracas, Ciudad de Buenos Aires, Argentina",7.6,77474,4,-58.387562,-34.645285,0.000000,0.010194,0.00
43,"Villa Real, Ciudad de Buenos Aires, Argentina",1.3,14278,10,-58.525877,-34.618943,0.000000,0.010983,0.00
39,"Villa Lugano, Ciudad de Buenos Aires, Argentina",9.0,114253,8,-58.477689,-34.674153,0.000000,0.012695,0.00
1,"Almagro, Ciudad de Buenos Aires, Argentina",4.1,139262,5,-58.422233,-34.609988,0.100000,0.033966,2.94
36,"Villa del Parque, Ciudad de Buenos Aires, Arge...",3.4,58573,11,-58.493821,-34.604797,0.076923,0.017227,4.47
12,"Floresta, Ciudad de Buenos Aires, Argentina",2.3,39473,10,-58.483791,-34.628105,0.076923,0.017162,4.48
32,"San Telmo, Ciudad de Buenos Aires, Argentina",1.2,25969,1,-58.373750,-34.621401,0.100000,0.021641,4.62
2,"Balvanera, Ciudad de Buenos Aires, Argentina",4.4,152198,3,-58.403140,-34.609215,0.160000,0.034590,4.63
30,"San Cristóbal, Ciudad de Buenos Aires, Argentina",2.1,49986,3,-58.402390,-34.624060,0.113636,0.023803,4.77


In [291]:
df.to_csv(r'C:\\Users\caika\OneDrive\Desktop\pYTHON\DATA LOCATION\bsascoffeefinal.csv', index = None, header=True)

#### Cleaning final table

In [293]:
df_final=df
df_final['Neighbourhoods'] = df_final['Neighbourhoods'].str.replace(', Ciudad de Buenos Aires, Argentina', '')
df_final.reset_index(inplace=True)
df_final.drop(['Area', 'Population', 'District', 'Longitude', 'Latitude','COFFEES' ], axis=1, inplace=True)
df_final

,index,Neighbourhoods,PopDensity,CoffeeDensity
0,19,Nueva Pompeya,0.010206,0.00
1,3,Barracas,0.010194,0.00
2,43,Villa Real,0.010983,0.00
3,39,Villa Lugano,0.012695,0.00
4,1,Almagro,0.033966,2.94
5,36,Villa del Parque,0.017227,4.47
6,12,Floresta,0.017162,4.48
7,32,San Telmo,0.021641,4.62
8,2,Balvanera,0.034590,4.63
9,30,San Cristóbal,0.023803,4.77


## END

#### Thanks for reading.